Ce modèle présente 3 stratégies qui permettent de réaliser une classification d'images avec un petit ensemble d'image - l'entraînement d'un petit modèle à partir de zéro, l'extraction de caractéristiques à l'aide d'un modèle pré-entraîné et le réglage de précision d'un modèle pré-entraîné

# Entraînement d'un petit modèle à partir de zéro

Il faut télécharger les données au préalable : 
`curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip`
`unzip -q kagglecatsanddogs_5340.zip`

Ensuite, nous allons séparer nos données en 3 ensembles (dossiers) : entraînement, validation, test

In [1]:
import os
import shutil

# Chemin vers le répertoire où vous avez stocké les données décompressées
original_dataset_dir = '/Users/ayoub/Documents/GitHub/Deep-learning/Chapter5/PetImages'

# Répertoire où vous stockerez votre petit ensemble de données
base_dir = '/Users/ayoub/Documents/GitHub/Deep-learning/Chapter5/cats_and_dogs_small'
os.mkdir(base_dir)

# Répertoires contenant les ensembles d'entraînement, de validation et de test
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# Répertoire avec les images de chats pour l'entraînement
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

# Répertoire avec les images de chiens pour l'entraînement
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

# Répertoire avec les images de chats pour la validation
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

# Répertoire avec les images de chiens pour la validation
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

# Répertoire avec les images de chats pour le test
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

# Répertoire avec les images de chiens pour le test
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

# Copie des 1000 premières images de chats dans le répertoire train_cats_dir
fnames = ['{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, 'Cat', fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

# Copie des 500 images suivantes de chats dans le répertoire validation_cats_dir
fnames = ['{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, 'Cat', fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)

# Copie des 500 images suivantes de chats dans le répertoire test_cats_dir
fnames = ['{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, 'Cat', fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)

# Copie des 1000 premières images de chiens dans le répertoire train_dogs_dir
fnames = ['{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, 'Dog', fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)

# Copie des 500 images suivantes de chiens dans le répertoire validation_dogs_dir
fnames = ['{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, 'Dog', fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)

# Copie des 500 images suivantes de chiens dans le répertoire test_dogs_dir
fnames = ['{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, 'Dog', fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)


In [2]:
print('total training cat images:', len(os.listdir(train_cats_dir)))

total training cat images: 1000


## On initialise un premier ConvNet : 

In [5]:
from keras import layers
from keras import models

model = models.Sequential([
    layers.Input(shape=(150, 150, 3)),  
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [8]:
# Compilation 

from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])

## Prétraitement des données

Convertir le format JPEG, en tenseurs contenant des floats. Puis transformer les valeurs des pixels en intervalle 0-1.

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Modifie l'échelle des valeurs des images en appliquant le coefficient 1/255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    # train_dir est le répertoire contenant les données d'entraînement
    train_dir,
    # target_size modifie la dimension des images afin qu'elles mesurent 150 × 150 pixels
    target_size=(150, 150),
    batch_size=20,
    # Nous avons besoin de labels binaires, car nous utilisons la fonction de perte binary_crossentropy
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [12]:
# Configuration du modèle avec un générateur de lots
history = model.fit(
    train_generator,
    steps_per_epoch=100,  
    epochs=30,            
    validation_data=validation_generator,
    validation_steps=50   
)


Epoch 1/30


/Users/ayoub/miniforge3/envs/keras/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 12/100 ━━━━━━━━━━━━━━━━━━━━ 9s 111ms/step - acc: 0.5159 - loss: 0.6921

2024-05-02 14:49:30.070627: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x2d17c2c20>
Traceback (most recent call last):

  File "/Users/ayoub/miniforge3/envs/keras/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/Users/ayoub/miniforge3/envs/keras/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Users/ayoub/miniforge3/envs/keras/lib/python3.9/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/ayoub/miniforge3/envs/keras/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py", line 250, in _get_iterator
    for i, batch in enumerate(gen_fn()):

  File "/Users/ayoub/miniforge3/envs/keras/lib/python3.9/sit

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x2d17c2c20>
Traceback (most recent call last):

  File "/Users/ayoub/miniforge3/envs/keras/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/Users/ayoub/miniforge3/envs/keras/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Users/ayoub/miniforge3/envs/keras/lib/python3.9/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/ayoub/miniforge3/envs/keras/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py", line 250, in _get_iterator
    for i, batch in enumerate(gen_fn()):

  File "/Users/ayou